# The Elite Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [1]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [2]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [3]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    """ pi: list with probability values
        return roulette proportions"""
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    """ ppi: list of roulette proportions
        Pop: list of corresponding item   [item1, ... , itemN] """
    
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost

In [4]:
# calculate cost of each route
def Cost(Pop, graph):
    """ Pop: List of All Items in the e.g [[item1], ... , [itemN]]
    graph: cost matrix or graph 
    returns a cost vector for corresponding item """
    
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += graph[route[j]][route[j+1]]
        cost += graph[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Initialize Pheromone 

In [5]:
# AS initialization
def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# EAS initialization
def EASTSPher(graph, e, m, rho):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    numer = e + m
    den = rho*cost
    pher = [[numer/den for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [6]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        
    return connections

# Find Path 

In [7]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Get Elite Route and Connections

In [8]:
def Elite(Elite, ant, graph):
    
    if Cost([ant], graph)[0] < Cost([Elite], graph)[0]:
        return ant
    else:
        return Elite

def connections(L):
    """L is a list
    one way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
    
    cons.append((L[-1], L[0]))
    
    return cons

def connectionsE(L):
    """L is a list
    Two way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
        cons.append( (L[i], L[i-1]) )
    
    cons.append((L[-1], L[0]))
    cons.append((L[0], L[1]))
    
    return cons

# Update Pheromone 

In [9]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix


def EASUpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        matrix[route[i]][route[i+1]] += tau 
        matrix[route[i+1]][route[i]] += tau 
        
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

def ElitePheromone(Elite, matrix, e):
    """route : route taken
    matrix: pheromone matrix"""
    
    bstau = 1/Cost([Elite], matrix)[0]
    #tau = 1
    
    for i in range(len(Elite)-1):
        matrix[Elite[i]][Elite[i+1]] += (e*bstau) 
        matrix[Elite[i+1]][Elite[i]] += (e*bstau) 
        
    matrix[Elite[-1]][Elite[0]] += (e*bstau)
    matrix[Elite[0]][Elite[-1]] += (e*bstau)

    return matrix

# Evaporate Pheromone 

In [10]:
def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [11]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [12]:
def AS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, e = 10, runs = 50, Pher = None):
    
    if Pher == None:
        #Pher = TSPher(Ngraph)
        Pher = EASTSPher(Ngraph, e, ants, rho)
        
    starthob = random.choice([i for i in range(len(Ngraph))])

    GOAT = [i for i in range(len(Ngraph))]  # initial best ant
    
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
            GOAT = Elite(GOAT, stof, Ngraph)
   
        #Pheromone update
        for j in Ants:
            EASUpPheromone(j, Pher)
            
        # Add Elite Pheromone
        ElitePheromone(GOAT, Pher, e)
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            E = Cost([GOAT], Ngraph)[0]
            print("Evolution ", i, " Elite Cost = ", E,  " Minimum Cost = ", min(C), "  and Maximum Cost = ", max(C))
            #print("<<<<<<<<  The Best Route IS  >>>>>>>> ", GOAT)
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [13]:
# Toy Network graph 
D = 100
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

np.array(graph)

array([[ 0,  2,  9, ...,  4,  7,  5],
       [ 2,  0,  5, ...,  8,  3,  4],
       [ 9,  5,  0, ...,  2,  8,  7],
       ...,
       [ 4,  8,  2, ...,  0, 10,  4],
       [ 7,  3,  8, ..., 10,  0,  3],
       [ 5,  4,  7, ...,  4,  3,  0]])

In [14]:
# Find Shortest Path with the Algorithm

ants = len(graph)
alpha = 1
beta = 3
rho = 0.5
e = len(graph)
runs = 200

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  270  Minimum Cost =  270   and Maximum Cost =  336
Evolution  10  Elite Cost =  206  Minimum Cost =  207   and Maximum Cost =  242
Evolution  20  Elite Cost =  204  Minimum Cost =  205   and Maximum Cost =  240
Evolution  30  Elite Cost =  203  Minimum Cost =  208   and Maximum Cost =  241
Evolution  40  Elite Cost =  203  Minimum Cost =  206   and Maximum Cost =  244
Evolution  50  Elite Cost =  203  Minimum Cost =  206   and Maximum Cost =  247
Evolution  60  Elite Cost =  203  Minimum Cost =  203   and Maximum Cost =  241
Evolution  70  Elite Cost =  202  Minimum Cost =  207   and Maximum Cost =  244
Evolution  80  Elite Cost =  202  Minimum Cost =  205   and Maximum Cost =  236
Evolution  90  Elite Cost =  202  Minimum Cost =  208   and Maximum Cost =  243
Evolution  100  Elite Cost =  201  Minimum Cost =  204   and Maximum Cost =  241
Evolution  110  Elite Cost =  201  Minimum Cost =  202   and Maximum Cost =  239
Evolution  120  Elite Cost =  200  Mini

(array([[26, 39, 11, ..., 54, 64, 13],
        [26,  2, 21, ..., 23, 10, 29],
        [26, 39, 55, ...,  5, 67, 32],
        ...,
        [26, 39, 55, ..., 84, 10, 76],
        [26, 39, 43, ...,  5, 60, 90],
        [26, 39, 55, ..., 91, 48,  5]]), 204)

# Soving Real TSP problems 

In [15]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 4
rho = 0.5
e = len(graph)
runs = 200

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  42189.12625839805  Minimum Cost =  42189.12625839805   and Maximum Cost =  61014.398025635965
Evolution  10  Elite Cost =  36813.604452509586  Minimum Cost =  36813.604452509586   and Maximum Cost =  50428.96640744907
Evolution  20  Elite Cost =  34767.29087768003  Minimum Cost =  34799.876556892654   and Maximum Cost =  51181.64610380037
Evolution  30  Elite Cost =  34767.29087768003  Minimum Cost =  35940.05227749613   and Maximum Cost =  49757.5499796695
Evolution  40  Elite Cost =  34767.29087768003  Minimum Cost =  35761.88091387343   and Maximum Cost =  51878.41565052144
Evolution  50  Elite Cost =  34767.29087768003  Minimum Cost =  35938.116400476756   and Maximum Cost =  49950.72682593462
Evolution  60  Elite Cost =  34767.29087768003  Minimum Cost =  34943.52636428335   and Maximum Cost =  48737.71307831741
Evolution  70  Elite Cost =  34767.29087768003  Minimum Cost =  34943.52636428335   and Maximum Cost =  57281.81723667205
Evolution  80  Elite 

(array([[46, 20, 12, ..., 23, 31, 38],
        [46, 20, 12, ..., 34, 44, 39],
        [46, 12, 20, ..., 44, 25,  3],
        ...,
        [46, 20, 12, ..., 23, 31, 38],
        [46, 12, 24, ..., 34, 25,  3],
        [46, 20, 12, ..., 38, 30, 43]]), 35240.959090946555)

In [16]:
# burma14

import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 4
rho = 0.5
e = len(graph)
runs = 1000

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  31.22691510942754  Minimum Cost =  31.22691510942754   and Maximum Cost =  51.671683978022834
Evolution  50  Elite Cost =  31.22691510942754  Minimum Cost =  33.84835846148926   and Maximum Cost =  42.153896936793274
Evolution  100  Elite Cost =  31.22691510942754  Minimum Cost =  36.1497482131453   and Maximum Cost =  36.782576829400035
Evolution  150  Elite Cost =  31.22691510942754  Minimum Cost =  31.22691510942754   and Maximum Cost =  42.39466805738795
Evolution  200  Elite Cost =  31.22691510942754  Minimum Cost =  31.22691510942754   and Maximum Cost =  36.782576829400035
Evolution  250  Elite Cost =  31.22691510942754  Minimum Cost =  36.782576829400035   and Maximum Cost =  40.95839624653566
Evolution  300  Elite Cost =  31.22691510942754  Minimum Cost =  31.22691510942754   and Maximum Cost =  39.647431683212986
Evolution  350  Elite Cost =  31.22691510942754  Minimum Cost =  34.47128513071587   and Maximum Cost =  39.647431683212986
Evolution  40

(array([[13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3,  4,  5, 11,  6, 12,  7,  0, 10,  8,  9,  1],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  1,  9,  8, 10,  0,  7, 12,  6, 11,  5,  4,  3,  2],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  1,  9,  8, 10,  0,  7, 12,  6,  5, 11,  3,  2,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4],
        [13,  2,  3, 11,  5,  6, 12,  7,  0, 10,  8,  9,  1,  4]]),
 31.22691510942754)